In [ ]:
# from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
data = shelter.read({})
df = pd.DataFrame.from_records(data)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME - DONE Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(id='hidden-div', style={'display':'none'}),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        html.Label('Filter by Rescue Type'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        )
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '0px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },
        style_data_conditional=[],
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },
        editable=True,
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        column_selectable='single',
        row_selectable='multi',
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        page_size=10, 
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'Reset':
        data = shelter.read({})
    else:
        rescue_breeds = {
            'Water Rescue': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
            'Mountain Rescue': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
            'Disaster Rescue': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        }

        query = {
            "breed": {"$in": rescue_breeds[filter_type]}
        }
        data = shelter.read(query)
    
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # Add error handling for data transformation
    try:
        data_df = pd.DataFrame.from_records(data)
        if '_id' in data_df.columns:
            data_df.drop(columns=['_id'], inplace=True)
        data_dict = data_df.to_dict('records')
        print(f"Data converted to dict: {len(data_dict)} records")
    except Exception as e:
        print(f"Error converting data to dict: {e}")
        data_dict = []

    print(f"Filter type: {filter_type}")
    print(f"Query: {query if filter_type != 'Reset' else 'Reset'}")
    print(f"Data: {data_dict[:5]}")  # Print the first 5 records for debugging
    
    return data_dict, columns




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75, -97.48]
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    if dff.empty or row >= len(dff):
        return [html.Div("No data available")]

    lat = dff.iloc[row]['location_lat'] if 'location_lat' in dff.columns and pd.notna(dff.iloc[row]['location_lat']) else 30.75
    lon = dff.iloc[row]['location_long'] if 'location_long' in dff.columns and pd.notna(dff.iloc[row]['location_long']) else -97.48
    
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed'] if 'breed' in dff.columns else "Unknown"),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'] if 'name' in dff.columns else "Unknown")
                ])
            ])
        ])
    ]

print("Running...")

# df
app.run_server(debug=True)